In [25]:
import os
import sys
import time
import json
from datetime import datetime
import requests
from fastapi.responses import JSONResponse
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv


load_dotenv(dotenv_path='/home/psor/testgit/test_project/get_stocks/.env')

STOCK_API_KEY = os.getenv('STOCK_API_KEY')
PROJECT_ID = os.environ.get('PROJECT_ID')  
# No need to explicitly provide credentials when running on Cloud Run. The BigQuery client will automatically use the service account associated with the Cloud Run service.
credentials = service_account.Credentials.from_service_account_file(
        "/mnt/c/Users/m_was/Downloads/tomastestproject-433206-48a55703dec2.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_id = f"{PROJECT_ID}.testdb_1.raw_stock_data"

def fetch_and_upload_raw_stock_data_daily(stock_symbol: str, API_KEY: str):
    try:
        # Construct the API URL
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={STOCK_API_KEY}"
        response = requests.get(url=url)   
        response.raise_for_status()
        data = response.json()

        # Extract the latest day's data
        latest_date = sorted(data['Time Series (Daily)'].keys())[-1]
        latest_data = data['Time Series (Daily)'][latest_date]

        fetch_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Prepare data for insertion
        rows_to_insert = [
            {
                "stock_symbol": stock_symbol,
                "raw_data": json.dumps(latest_data), # Store only latest data
                "fetch_date": fetch_date  
            }
        ]
        
        # Use upsert to avoid duplicates based on (stock_symbol, trade_date)
        job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND", # or WRITE_APPEND if you want to keep historical data
       )

        load_job = client.load_table_from_json(rows_to_insert, table_id, job_config=job_config) #job_config=job_config)
        load_job.result()  # Wait for the job to complete.

        if load_job.errors:
            print(f"Encountered errors while inserting rows: {load_job.errors}")
        else:
            print("Rows successfully inserted.")
        
    except Exception as e:
            print(f"An error occurred: {e}")


#if __name__ == "__main__":

In [26]:
fetch_and_upload_raw_stock_data_daily('TSLA', STOCK_API_KEY)

Rows successfully inserted.
